In [ ]:
from pathlib import Path

import torch
import cv2 as cv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display

import torchvision.transforms as T


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
video_name = '/content/drive/MyDrive/Colab Notebooks/piv_video/re1000raw500frame2.avi' # or any other extension like .avi etc
vidcap = cv2.VideoCapture(video_name)
success,image = vidcap.read()
count = 1
while success:
  cv2.imwrite("%d.jpg" % count, image)     # save frame as JPEG file
  success,image = vidcap.read()
  count += 1

In [ ]:
import getopt
import math
import numpy
import PIL
import PIL.Image
import sys
import torch
import tensorflow as tf
##########################################################

assert(int(str('').join(torch.__version__.split('.')[0:2])) >= 13) # requires at least pytorch version 1.3.0

torch.set_grad_enabled(False) # make sure to not compute gradients for computational performance

torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance

##########################################################

arguments_strModel = 'sintel-final' # 'sintel-final', or 'sintel-clean', or 'chairs-final', or 'chairs-clean', or 'kitti-final'
arguments_strOne = '0.png'
arguments_strTwo = '1.png'
arguments_strOut = 'out.png'



##########################################################

backwarp_tenGrid = {}

def backwarp(tenInput, tenFlow):
    if str(tenFlow.shape) not in backwarp_tenGrid:
        tenHor = torch.linspace(-1.0 + (1.0 / tenFlow.shape[3]), 1.0 - (1.0 / tenFlow.shape[3]), tenFlow.shape[3]).view(1, 1, 1, -1).repeat(1, 1, tenFlow.shape[2], 1)
        tenVer = torch.linspace(-1.0 + (1.0 / tenFlow.shape[2]), 1.0 - (1.0 / tenFlow.shape[2]), tenFlow.shape[2]).view(1, 1, -1, 1).repeat(1, 1, 1, tenFlow.shape[3])

        backwarp_tenGrid[str(tenFlow.shape)] = torch.cat([ tenHor, tenVer ], 1)
    # end

    tenFlow = torch.cat([ tenFlow[:, 0:1, :, :] / ((tenInput.shape[3] - 1.0) / 2.0), tenFlow[:, 1:2, :, :] / ((tenInput.shape[2] - 1.0) / 2.0) ], 1)

    return torch.nn.functional.grid_sample(input=tenInput, grid=(backwarp_tenGrid[str(tenFlow.shape)] + tenFlow).permute(0, 2, 3, 1), mode='bilinear', padding_mode='border', align_corners=False)
# end

##########################################################

class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()

        class Preprocess(torch.nn.Module):
            def __init__(self):
                super().__init__()
            # end

            def forward(self, tenInput):
                tenInput = tenInput.flip([1])
                tenInput = tenInput - torch.tensor(data=[0.485, 0.456, 0.406], dtype=tenInput.dtype, device=tenInput.device).view(1, 3, 1, 1)
                tenInput = tenInput * torch.tensor(data=[1.0 / 0.229, 1.0 / 0.224, 1.0 / 0.225], dtype=tenInput.dtype, device=tenInput.device).view(1, 3, 1, 1)

                return tenInput
            # end
        # end

        class Basic(torch.nn.Module):
            def __init__(self, intLevel):
                super().__init__()

                self.netBasic = torch.nn.Sequential(
                    torch.nn.Conv2d(in_channels=8, out_channels=32, kernel_size=7, stride=1, padding=3),
                    torch.nn.ReLU(inplace=False),
                    torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7, stride=1, padding=3),
                    torch.nn.ReLU(inplace=False),
                    torch.nn.Conv2d(in_channels=64, out_channels=32, kernel_size=7, stride=1, padding=3),
                    torch.nn.ReLU(inplace=False),
                    torch.nn.Conv2d(in_channels=32, out_channels=16, kernel_size=7, stride=1, padding=3),
                    torch.nn.ReLU(inplace=False),
                    torch.nn.Conv2d(in_channels=16, out_channels=2, kernel_size=7, stride=1, padding=3)
                )
            # end

            def forward(self, tenInput):
                return self.netBasic(tenInput)
            # end
        # end

        self.netPreprocess = Preprocess()

        self.netBasic = torch.nn.ModuleList([ Basic(intLevel) for intLevel in range(6) ])

        self.load_state_dict({ strKey.replace('module', 'net'): tenWeight for strKey, tenWeight in torch.hub.load_state_dict_from_url(url='http://content.sniklaus.com/github/pytorch-spynet/network-' + arguments_strModel + '.pytorch', file_name='spynet-' + arguments_strModel).items() })
    # end

    def forward(self, tenOne, tenTwo):
        tenFlow = []

        tenOne = [ self.netPreprocess(tenOne) ]
        tenTwo = [ self.netPreprocess(tenTwo) ]

        for intLevel in range(1):
            if tenOne[0].shape[2] > 128 or tenOne[0].shape[3] >  128:
                tenOne.insert(0, torch.nn.functional.avg_pool2d(input=tenOne[0], kernel_size=2, stride=2, count_include_pad=False))
                tenTwo.insert(0, torch.nn.functional.avg_pool2d(input=tenTwo[0], kernel_size=2, stride=2, count_include_pad=False))
            # end
        # end

        tenFlow = tenOne[0].new_zeros([ tenOne[0].shape[0], 2, int(math.floor(tenOne[0].shape[2] / 2.0)), int(math.floor(tenOne[0].shape[3] / 2.0)) ])

        for intLevel in range(len(tenOne)):
            tenUpsampled = torch.nn.functional.interpolate(input=tenFlow, scale_factor=2, mode='bilinear', align_corners=True) * 2.0

            if tenUpsampled.shape[2] != tenOne[intLevel].shape[2]: tenUpsampled = torch.nn.functional.pad(input=tenUpsampled, pad=[ 0, 0, 0, 1 ], mode='replicate')
            if tenUpsampled.shape[3] != tenOne[intLevel].shape[3]: tenUpsampled = torch.nn.functional.pad(input=tenUpsampled, pad=[ 0, 1, 0, 0 ], mode='replicate')

            tenFlow = self.netBasic[intLevel](torch.cat([ tenOne[intLevel], backwarp(tenInput=tenTwo[intLevel], tenFlow=tenUpsampled), tenUpsampled ], 1)) + tenUpsampled
        # end

        return tenFlow
    # end
# end

netNetwork = None

##########################################################

def estimate(tenOne, tenTwo):
    global netNetwork
    if netNetwork is None:
        netNetwork = Network().eval()


    assert(tenOne.shape[1] == tenTwo.shape[1])
    assert(tenOne.shape[2] == tenTwo.shape[2])

    intWidth = tenOne.shape[2]
    intHeight = tenOne.shape[1]


    #assert(intWidth == 1368) # remember that there is no guarantee for correctness, comment this line out if you acknowledge this and want to continue
    #assert(intHeight == 2280) # remember that there is no guarantee for correctness, comment this line out if you acknowledge this and want to continue

    tenPreprocessedOne = tenOne.view(1, 3, intHeight, intWidth)
    tenPreprocessedTwo = tenTwo.view(1, 3, intHeight, intWidth)


    intPreprocessedWidth = int(math.floor(math.ceil(intWidth / 32.0) * 32.0))
    intPreprocessedHeight = int(math.floor(math.ceil(intHeight / 32.0) * 32.0))


    tenPreprocessedOne = torch.nn.functional.interpolate(input=tenPreprocessedOne, size=(intPreprocessedHeight, intPreprocessedWidth), mode='bilinear', align_corners=False)
    tenPreprocessedTwo = torch.nn.functional.interpolate(input=tenPreprocessedTwo, size=(intPreprocessedHeight, intPreprocessedWidth), mode='bilinear', align_corners=False)

    tenFlow = torch.nn.functional.interpolate(input=netNetwork(tenPreprocessedOne, tenPreprocessedTwo), size=(intHeight, intWidth), mode='bilinear', align_corners=False)

    tenFlow[:, 0, :, :] *= float(intWidth) / float(intPreprocessedWidth)
    tenFlow[:, 1, :, :] *= float(intHeight) / float(intPreprocessedHeight)

    return tenFlow[0, :, :, :].cpu()
# end

##########################################################

if __name__ == '__main__':
    step = 13
    count1 = 1
    count2 = 2
    width = 1024
    height = 591
    tensor1 = np.zeros((2,width,height))
    tensor2 = np.zeros((2,width,height))
    from PIL import ImageOps
    for i in range(1,30):
        img = Image.open(f'{count1}.png')
        imgGray1 = numpy.array(img)

        img = Image.open(f'{count2}.png')
        imgGray2 = numpy.array(img)

        # Convert grayscale images to RGB
        imgGray1_rgb = ImageOps.grayscale(Image.fromarray(imgGray1)).convert('RGB')
        imgGray2_rgb = ImageOps.grayscale(Image.fromarray(imgGray2)).convert('RGB')

        # Convert to float and normalize
        imgGray1_rgb = torch.FloatTensor(numpy.array(imgGray1_rgb).astype(numpy.float32) * (1.0 / 255.0))
        imgGray2_rgb = torch.FloatTensor(numpy.array(imgGray2_rgb).astype(numpy.float32) * (1.0 / 255.0))

        # Add batch dimension
        tenOne = imgGray1_rgb.unsqueeze(0)
        tenTwo = imgGray2_rgb.unsqueeze(0)


        # Process the tensors
        tenOutput = estimate(tenOne, tenTwo)


        max_x = np.max(tenOutput.numpy()[0,:,:])
        norm_tensor_x = tenOutput.numpy()[0,:,:]/(max_x*32)
        max_y = np.max(tenOutput.numpy()[1,:,:])
        norm_tensor_y = tenOutput.numpy()[1,:,:]*(-1)/(max_y*32)
        tensor1[0,:,:] +=norm_tensor_x
        tensor1[1,:,:] +=norm_tensor_y
        tensor5 = tenOutput.numpy()

        count1 = count1 + 1
        count2 = count2 + 1
        print(count1)


    tensor1 = tensor1
    tensor2 = tensor1/29
    print(tenOutput.shape)

    from torchvision.utils import save_image
    tenOutput1 = tenOutput.numpy()

    plt.quiver(np.arange(0, height, step), np.arange(width, -1, -step),
          tensor1[0,::step, ::step], tensor1[1,::step, ::step])
    plt.axis('off')
    plt.gcf().set_size_inches(128, 128)


In [ ]:
import seaborn as sns
import cv2
mag_img, pha_img = cv2.cartToPolar(tensor5[1, ...], tensor5[0, ...])

im = sns.heatmap(mag_img[:,:], cmap="jet",cbar=True,annot_kws={"size": 16})
sns.set(font_scale=2)


plt.gcf().set_size_inches(64, 32)
sns.set(font_scale=10)
plt.savefig("mag.tif")